In [2]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split


## 1 : Yes (Y)
## 2 : Abstain (A)
## 3 : No (N)
## 8 : Non-participating
## 9 : Not eligible to participate



# Objective 3. Using different time periods to predict resolutions

In [3]:
# votes_df = votes_df.loc[votes_df['state_name'] == 'United States of America']

# Removing all abstain votes for now https://www.kaggle.com/datasets/unitednations/general-assembly/discussion/29187
# votes_df = votes_df.loc[votes_df["vote"] != 2]

votes_df_2 = os.path.join("undataset2", 'UNVotes-1.csv')
votes_df_2 = pd.read_csv(votes_df_2, encoding='latin-1')


votes_df_2 = votes_df_2.loc[votes_df_2['Countryname'] == 'United States of America']
votes_df_2 = votes_df_2.loc[(votes_df_2["vote"] != 2 )]
votes_df_2 = votes_df_2.loc[(votes_df_2["vote"] != 8 )]
votes_df_2 = votes_df_2.loc[(votes_df_2["vote"] != 9 )]

# mask = (votes_df_2['year'] >= 1981) & (votes_df_2['year'] <= 1991)
# votes_df_2 = votes_df_2.loc[mask]

# print(votes_df_2['vote'].value_counts())

# ammend = votes_df_2[['descr','vote']]

# check_for_nan = ammend['descr'].isnull()
# print (type(ammend['descr'].iloc[0]))

# 2009 - 2013
# 3    240
# 1    128

# 2017 - 2021
# 3    146
# 1     55

C:\Users\aidan\AppData\Local\Temp\ipykernel_40980\3942605212.py:7: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  votes_df_2 = pd.read_csv(votes_df_2, encoding='latin-1')


In [4]:
### import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.metrics import classification_report


# Load the dataset

df = votes_df_2
train = df.loc[(df['year'] >= 2009) & (df['year'] <= 2013)]
test = df.loc[(df['year'] >= 2018)]
print(len(train))
print(len(test))

X_train = train['descr']
X_test = test['descr']
y_train = train['vote']
y_test = test['vote']


# Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df['descr'], df['vote'], test_size=0.15, random_state=27)

# X = pd.concat([X_train, y_train], axis=1)
# yes = X[X.vote==1]
# no = X[X.vote==3]

# df_yes = yes
# df_no = no
# print(len(df_yes), len(df_no))

# if (len(df_yes) > len(df_no)):
#     print("Yes has more votes than no")
#     df_sampled = resample(df_yes, 
#                                      replace=False,     # sample with replacement
#                                      n_samples=len(df_no),    # to match minority class
#                                      random_state=27) # reproducible results
#     sampled = pd.concat([df_sampled, df_no])
    
# if(len(df_no) > len(df_yes)):
#     print("No has more votes than yes")
#     df_sampled = resample(df_no, 
#                                      replace=False,     # sample with replacement
#                                      n_samples=len(df_yes),    # to match minority class
#                                      random_state=27) # reproducible results
#     sampled = pd.concat([df_yes, df_sampled])


# no_sampled = sampled.loc[(sampled["vote"] == 3)]

# pd.options.display.max_colwidth = 500
# print(no_sampled.iloc[41])
# print(len(sampled))

# y_train = sampled.vote
# X_train = sampled.descr

# print(len(y_train), len(X_train))


# Vectorize the text data

vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train.values.astype('U').ravel())
X_test_vectors = vectorizer.transform(X_test.values.astype('U').ravel())

# Train the  Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_vectors, y_train)

# Evaluate the performance of the classifier
y_pred = clf.predict(X_test_vectors)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(y_pred.tolist().count(1), y_test.tolist().count(1))
print(y_pred.tolist().count(3), y_test.tolist().count(3))
print(y_pred.tolist().count(2), y_test.tolist().count(2))
print(y_pred.tolist().count(8), y_test.tolist().count(8))
print(y_pred.tolist().count(9), y_test.tolist().count(9))

print("\t\t\t Naive Bayes report:\n",classification_report(y_pred,y_test))

368
85
Accuracy: 0.8235294117647058
8 19
77 66
0 0
0 0
0 0
			 Naive Bayes report:
               precision    recall  f1-score   support

           1       0.32      0.75      0.44         8
           3       0.97      0.83      0.90        77

    accuracy                           0.82        85
   macro avg       0.64      0.79      0.67        85
weighted avg       0.91      0.82      0.85        85



# Logistic Regression

In [5]:

from sklearn.linear_model import LogisticRegression
reg = LogisticRegression(class_weight='balanced')
reg.fit(X_train_vectors, y_train)
y_pred_lr = reg.predict(X_test_vectors)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy_lr)
print(y_pred_lr.tolist().count(1), y_test.tolist().count(1))
print("\t\t\t Logistic Regression report:\n",classification_report(y_pred_lr,y_test))

Accuracy: 0.788235294117647
17 19
			 Logistic Regression report:
               precision    recall  f1-score   support

           1       0.47      0.53      0.50        17
           3       0.88      0.85      0.87        68

    accuracy                           0.79        85
   macro avg       0.68      0.69      0.68        85
weighted avg       0.80      0.79      0.79        85



# Decision Tree

In [6]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train_vectors, y_train)
y_pred_dt = dtc.predict(X_test_vectors)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Accuracy:", accuracy_dt)
print(y_pred_dt.tolist().count(3), y_test.tolist().count(3))
print("\t\t\t Decision Tree report:\n",classification_report(y_pred_dt,y_test))

Accuracy: 0.7647058823529411
60 66
			 Decision Tree report:
               precision    recall  f1-score   support

           1       0.63      0.48      0.55        25
           3       0.80      0.88      0.84        60

    accuracy                           0.76        85
   macro avg       0.72      0.68      0.69        85
weighted avg       0.75      0.76      0.75        85



# Random Forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train_vectors, y_train)
y_pred_rf = rfc.predict(X_test_vectors)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy:", accuracy_rf)
print(y_pred_rf.tolist().count(3), y_test.tolist().count(3))
print("\t\t\t Random Forest report:\n",classification_report(y_pred_rf,y_test))

Accuracy: 0.8588235294117647
72 66
			 Random Forest report:
               precision    recall  f1-score   support

           1       0.53      0.77      0.62        13
           3       0.95      0.88      0.91        72

    accuracy                           0.86        85
   macro avg       0.74      0.82      0.77        85
weighted avg       0.89      0.86      0.87        85



# SVM

In [8]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train_vectors, y_train)
y_pred_svm = svc.predict(X_test_vectors)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Accuracy:", accuracy_svm)
print(y_pred_svm.tolist().count(3), y_test.tolist().count(3))
print("\t\t\t SVM report:\n",classification_report(y_pred_svm,y_test))

Accuracy: 0.8117647058823529
74 66
			 SVM report:
               precision    recall  f1-score   support

           1       0.37      0.64      0.47        11
           3       0.94      0.84      0.89        74

    accuracy                           0.81        85
   macro avg       0.65      0.74      0.68        85
weighted avg       0.87      0.81      0.83        85



# K Neighbours

In [9]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train_vectors, y_train)
y_pred_knn = knn.predict(X_test_vectors)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print("Accuracy:", accuracy_knn)
print(y_pred_knn.tolist().count(3), y_test.tolist().count(3))
print("\t\t\t KNN report:\n",classification_report(y_pred_knn,y_test))

Accuracy: 0.8352941176470589
72 66
			 KNN report:
               precision    recall  f1-score   support

           1       0.47      0.69      0.56        13
           3       0.94      0.86      0.90        72

    accuracy                           0.84        85
   macro avg       0.71      0.78      0.73        85
weighted avg       0.87      0.84      0.85        85



C:\Users\aidan\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [10]:
list1 = [y_pred,y_pred_lr,y_pred_dt,y_pred_rf,y_pred_svm, y_pred_knn]
d =['NaiveBayes','LogisticRegression','DecisionTree','RandomForest','SVM', 'KNN']
accuracies={} 
k=0
list2 = []
for i in list1:
    list2.append(accuracy_score(i,y_test)*100)
for i in d:
    accuracies[i] = list2[k]
    k+=1

print("All Accuracies: ", accuracies)
print("The most accurate model is: ", max(accuracies, key=accuracies.get))

All Accuracies:  {'NaiveBayes': 82.35294117647058, 'LogisticRegression': 78.82352941176471, 'DecisionTree': 76.47058823529412, 'RandomForest': 85.88235294117646, 'SVM': 81.17647058823529, 'KNN': 83.52941176470588}
The most accurate model is:  RandomForest
